In [1]:
import os
import sys
src_path = sys.path[0].replace("scripts", "src")
if src_path not in sys.path:
    sys.path.append(src_path)
out_path = sys.path[0].replace("scripts", "output")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

from grid_manager import MPC_op
from data_loader import UCSD_dataloader
from battery_model import Battery_base
from optimizer import Battery_optimizer
from predictor import *
from exp_manager import ExperimentManager
import copy

In [2]:
exp_suffix = "Mar2023"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
assert os.path.exists(exp_folder)

In [3]:
bat_params_sample ={
    "bat_capacity": None,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

op_params_sample = {"K": 96,
            "dc_price": 0.6,
            "ev_efficacy": 0.98,
            "energy_price_sell": 0.6, 
            "deg_model_opt": "rainflow",
            "ev_charge_rule": "flex",
            "ev_charge_rule_default": "unif",
            "p_grid_max": "1.5",}


In [4]:
class Size_TCO_rep_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "strategy", "B_kWh", "deg_model_opt", "deg_model", "ev_charge_rule"

        op_params = copy.deepcopy(op_params_sample)
        bat_params = copy.deepcopy(bat_params_sample)
        op_params["deg_model_opt"] = params["deg_model_opt"]
        op_params["ev_charge_rule"] = params["ev_charge_rule"]
        bat_params["deg_model"] = params["deg_model"]
        
        bat_params["bat_capacity"] = params["B_kWh"]

        strategy = params["strategy"]
      

        mpc = MPC_op()

        # Step 1: load data
        mpc.load_data(loader=UCSD_dataloader, 
            tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
            bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=None, Pmax=10)

        # Step 2: Load historical data
        mpc.init_historical_data(loader=UCSD_dataloader,
            tstart=datetime(2019,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
            bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=None, Pmax=10)

        # Step 3: specify other operational params
        optimizer_params = {"strategy": strategy, "language":"gurobi"}

        mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

        # Step 4: specify battery
        mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

        # Step 5: initialize predictor
        # [Yi, 2023/03/08] modify predictor def
        mpc.init_predictor(shortcut="GT")

        # Step 6: initialize save_config
        mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
            folder_path=self.save_path,
            checkpoints="90D", recovery=True, recovery_from=None,
            )

        mpc.run(tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), exe_K=96, save=True)
        
        stats = dict(mpc.summary["All"])
        print("="*10, strategy, params["B_hr"], stats["TCO"], "="*10)
        return stats

In [5]:
log_fn = os.path.join(exp_folder, "LOG-Size_TCO_365.xlsx")
save_path = os.path.join(exp_folder, "EXP-size_tco_curve")
em = Size_TCO_rep_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="365_Size_TCO")
var_keys = ["strategy", "B_hr", "B_kWh", "deg_model_opt", "deg_model", "ev_charge_rule"]
em.run(keys=var_keys, num_trials=72)

========================= 2019-05-09 00:00:00 =========================
========================= 2019-05-10 00:00:00 =========================
========================= 2019-05-11 00:00:00 =========================
========================= 2019-05-12 00:00:00 =========================
========================= 2019-05-13 00:00:00 =========================
========================= 2019-05-14 00:00:00 =========================
========================= 2019-05-15 00:00:00 =========================
========================= 2019-05-16 00:00:00 =========================


e:\jerry ( 2021.1.20拷贝 )\Berkeley\Github Repo\Battery_sizing\src\optimizer.py:1019: UserWarning: EV demand infeas: s=0, idx=[0]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-05-17 00:00:00 =========================
========================= 2019-05-18 00:00:00 =========================


e:\jerry ( 2021.1.20拷贝 )\Berkeley\Github Repo\Battery_sizing\src\optimizer.py:1019: UserWarning: EV demand infeas: s=0, idx=[0]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-05-19 00:00:00 =========================
========================= 2019-05-20 00:00:00 =========================


...

========================= 2019-11-20 00:00:00 =========================
========================= 2019-11-21 00:00:00 =========================
========================= 2019-11-22 00:00:00 =========================
========================= 2019-11-23 00:00:00 =========================
========================= 2019-11-24 00:00:00 =========================
========================= 2019-11-25 00:00:00 =========================
========================= 2019-11-26 00:00:00 =========================


e:\jerry ( 2021.1.20拷贝 )\Berkeley\Github Repo\Battery_sizing\src\optimizer.py:1019: UserWarning: EV demand infeas: s=0, idx=[0]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-11-27 00:00:00 =========================
========================= 2019-11-28 00:00:00 =========================
========================= 2019-11-29 00:00:00 =========================


e:\jerry ( 2021.1.20拷贝 )\Berkeley\Github Repo\Battery_sizing\src\optimizer.py:1019: UserWarning: EV demand infeas: s=0, idx=[0]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-11-30 00:00:00 =========================
========================= 2019-12-01 00:00:00 =========================
========================= 2019-12-02 00:00:00 =========================
========================= 2019-12-03 00:00:00 =========================
========================= 2019-12-04 00:00:00 =========================
========================= 2019-12-05 00:00:00 =========================
========================= 2019-12-06 00:00:00 =========================
========================= 2019-12-07 00:00:00 =========================
========================= 2019-12-08 00:00:00 =========================
========================= 2019-12-09 00:00:00 =========================
========================= 2019-12-10 00:00:00 =========================
========================= 2019-12-11 00:00:00 =========================
========================= 2019-12-12 00:00:00 =========================
========================= 2019-12-13 00:00:00 ==================